# Get Ready for the Libraries

In [ ]:
!pip install catboost
!pip install yfinance
!pip install seaborn
!pip install PyPortfolioOpt

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

from math import exp
from datetime import date,timedelta
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt import black_litterman, plotting, objective_functions
from pypfopt.black_litterman import BlackLittermanModel
from pypfopt.efficient_frontier import EfficientFrontier

# Get Ready for the datasets

In [ ]:
!gdown --id 1qLoKEZHEjqvjgBB1CFX62oh7IGJptAk7
!gdown --id 1YZmaQNzgpkj-DHbyFUhzzSDXKajabViF
!gdown --id 1KRYPKo4ZEZdbq0RCmIutu6n93FYAjQDN

# Import data from Yahoo Finance

In [ ]:
#list of stock codes of S&P stocks
tickers_csv = pd.read_csv("S&P500.csv")
tickers = list(tickers_csv["Symbol"])

#eliminate stocks that are too new and have insuffcient traning data
bug_stocks = ["BRK.B","BF.B","EMBC","CEG","OGN","CARR","OTIS","CTVA","MRNA","FOX","FOXA","DOW","CDAY","IR"]
tickers = list(set(tickers) - set(bug_stocks))

In [ ]:
# Get AdjClose, Close, High, Low, Open, Volume
yf.pdr_override()
end = pd.to_datetime("30/04/2022")
start = end - timedelta(days=5000)
data = yf.download(tickers, start=start, end=end,group_by="ticker", interval="1d")

data_to_process = pd.DataFrame.copy(data)
data_processed = data_to_process.dropna(axis=0)
data_processed

In [ ]:
#Get information on SnP for comparison
SnP_start=data_processed.index[0].date()
SnP_end=date.today()
SnP=yf.download('^GSPC',start=SnP_start, end=SnP_end, interval='1d')

#Calculate log daily log return of S&P
SnP['SnP Log Return']=np.log(SnP['Close'])-np.log(SnP['Close']).shift(1)
SnP

# Feature Engineering

In [ ]:
def find_std(data,period,start_date):

    df = data.copy()
    end = df.index.get_loc(start_date)
    start=end-period
    if start>=0:
        log_return=df['Log Return'].iloc[start:end]
        std=np.std(log_return, ddof=1)
        return std
    else:
        return np.NaN


def find_beta(data,period,start_date):

    df = data.copy()
    end=df.index.get_loc(start_date)
    start=end-period
    if start>=0:
        log_return=df['Log Return'].iloc[start:end].to_numpy()
        SnP_log_return=df['SnP Log Return'].iloc[start:end].to_numpy().reshape(-1,1)

        regr = LinearRegression()
        beta=regr.fit(SnP_log_return,log_return).coef_[0]
        return beta
    else:
        return np.NaN

def ShiftNum(var):
    if var[1]=='D':
        return int(var[0])
    elif var[1]=='W':
        return int(var[0])*5
    elif var[1]=='M':
        return int(var[0])*20
    elif var[1]=='Y':
        return int(var[0])*250
    else:
        return("Please give a str whose format is xD, xW, xM, xY.")

In [ ]:
stocks={}
return_lag=['1D','3D','1W','2W','3W','1M','6W','2M','3M']
for stock in tickers:
    stocks[stock]=data_processed[stock][['Close']].copy()

    #retain stock code as a feature
    stocks[stock]['Stock']=[stock]*stocks[stock].shape[0]

    #daily log return of stocks
    stocks[stock]['Log Return']=np.log(stocks[stock]['Close'])-np.log(stocks[stock]['Close']).shift(1)

    #join with SnP return for generating other features 
    #(SnP return on that day will not be used as feature to avoid data leakage)
    stocks[stock]=stocks[stock].join(SnP[['SnP Log Return']], how='left')

    #SnP return of yesterday as a feature
    stocks[stock]['SnP Log Return_1D']=stocks[stock]['SnP Log Return'].shift(1)
    stocks[stock].dropna(inplace=True)

    #calculate s.d. of stock return in the past 30 days
    stocks[stock]['Volatility']=[ find_std(stocks[stock],30,date) for date in stocks[stock].index]

    #calculate beta(a measure of a stock's volatility in relation to the overall market) for the past 30days
    stocks[stock]['Beta']=[ find_beta(stocks[stock],30,date) for date in stocks[stock].index]

    #Use return from n_days ago as feature
    for var in return_lag:
        name = 'Return_' + var
        stocks[stock][name] =  stocks[stock]['Log Return'].shift(ShiftNum(var))

In [ ]:
#join information of all stocks in one df
cleaned=pd.DataFrame()
for stock in tickers:
    cleaned=pd.concat([cleaned,stocks[stock]])
cleaned.dropna(inplace=True)

# Data Processing

In [ ]:
# Read csv Files
riskfree = pd.read_csv('risk_free.csv')
market_cap = pd.read_csv("market_cap.csv").astype("int64")

# For cleaned
cleaned.index = pd.to_datetime(cleaned.index)
SnP_Return = cleaned[["SnP Log Return"]]
cleaned = cleaned.drop(columns=["SnP Log Return","Close"],axis=1)
cleaned = cleaned.sort_values(by=["Stock","Date"])

# For Market Capitalisation
market_cap = market_cap.iloc[0]
market_cap = market_cap.to_dict()

In [ ]:
# Get list[date] for later looping
date_list = cleaned.index
date_list = date_list.drop_duplicates()
stock_list = cleaned["Stock"]
stock_list = stock_list.drop_duplicates()

In [ ]:
SnP_Return = SnP_Return.drop_duplicates()
SnP_Return = SnP_Return.reset_index().drop_duplicates().set_index("Date")

In [ ]:
log_return_df = cleaned[['Stock','Log Return']].reset_index().set_index(['Stock','Date']).unstack(level=[0])
log_return_df = log_return_df["Log Return"]

In [ ]:
# Import SPY Data from API
SPY = yf.download('SPY', start=date_list[0], end=date_list[-1])

# Main Body for Machine Learning & Optimisation

In [ ]:
# Adjustible parameters
n_days=252
training_len=100

# Pre-set parameters
trade_days=date_list[-n_days:]
daily_return = []
ms_daily_return = []
mv_daily_return = []
seed=0

# Looping
for trade_day in trade_days:
    print(trade_day)
    trade_day_index=date_list.get_loc(trade_day)
    first_training_day_index=trade_day_index-training_len
    train_valid_dates=date_list[first_training_day_index:trade_day_index]
    
    # Train Validation Split
    train_days, eval_days=train_test_split(train_valid_dates,test_size=0.7,random_state=seed)
    seed+=1

    train_data=cleaned.loc[train_days,:]
    eval_data=cleaned.loc[eval_days,:]
    trade_day_data = cleaned.loc[trade_day]

    train_x=train_data.drop(['Log Return'],axis=1)
    train_y=train_data['Log Return']
    eval_x=eval_data.drop(['Log Return'],axis=1)
    eval_y=eval_data['Log Return']
    trade_day_x=trade_day_data.drop(['Log Return'],axis=1)
    trade_day_y=trade_day_data['Log Return']

    # Establish CatBoost Model 
    model=CatBoostRegressor(iterations=100,task_type="CPU",learning_rate=0.1,
                depth=8,l2_leaf_reg=0.0000001,allow_writing_files=False,
                eval_metric='MAPE',random_seed=0,thread_count=-1)
    eval_set=Pool(eval_x,eval_y,cat_features=[0])

    catboost_train_data = Pool(data=train_x,label=train_y,cat_features=[0])
    model.fit(catboost_train_data,
              eval_set=eval_set,
              plot=False,
              early_stopping_rounds=10)
    
    # Prediction from CatBoost Model
    preds_log_return=model.predict(trade_day_x)

    # Append Predicted Array to Log Return DataFrame
    temp_df = pd.DataFrame(preds_log_return).transpose()
    temp_df.columns = stock_list
    log_return_opt_df = log_return_df[first_training_day_index:trade_day_index]
    log_return_opt_df = log_return_opt_df.append(temp_df)
    log_return_opt_df.index = date_list[first_training_day_index:trade_day_index+1]

    # Ready for Optimisation
    portfolio = log_return_opt_df.applymap(lambda x: exp(x))
    cs_actual = CovarianceShrinkage(portfolio, frequency=len(log_return_opt_df))
    e_cov = cs_actual.ledoit_wolf()

    # Prepare Variables for Black Litterman Model
    market_prices = SPY.loc[log_return_opt_df.index[0]:log_return_opt_df.index[-2]]
    annual_risk_free = riskfree.loc[trade_day_index]['Price']/100
    daily_risk_free = (1+annual_risk_free)**(1/252)-1
    delta = black_litterman.market_implied_risk_aversion(market_prices['Close'], risk_free_rate=daily_risk_free)
    prior = black_litterman.market_implied_prior_returns(market_cap, delta, e_cov)
    viewdict={trade_day_x['Stock'].values[i]:exp(preds_log_return[i])-1 for i in range(len(preds_log_return))}
    bl = BlackLittermanModel(e_cov, pi=prior, absolute_views=viewdict)
    # Posterior estimate of returns
    ret_bl = bl.bl_returns()
    S_bl = bl.bl_cov()

    # Perform Maximum Sharpe Ratio Portfolio
    ms_ef = EfficientFrontier(ret_bl, S_bl,verbose=False)
    ms_ef.add_objective(objective_functions.L2_reg)

    # Decide Method of Optimisation by Error
    try:
      ms_ef.max_sharpe()
      ms_weights = ms_ef.clean_weights()
    except:
      ms_weights = ms_ef.nonconvex_objective(objective_functions.sharpe_ratio,
                          objective_args=(ms_ef.expected_returns, ms_ef.cov_matrix),
                          weights_sum_to_one=True,)
    ms_weights = list(ms_weights.values())
    ms_expected_return = ms_weights@temp_df.iloc[0].to_numpy().T
    ms_return = ms_weights@trade_day_y.to_numpy().T
    ms_daily_return = np.append(ms_daily_return,ms_return)

    # Perform Minimum Volatility Portfolio
    mv_ef = EfficientFrontier(ret_bl, S_bl)
    mv_ef.add_objective(objective_functions.L2_reg)
    mv_ef.min_volatility()
    mv_weights = mv_ef.clean_weights()

    mv_weights = list(mv_weights.values())
    mv_expected_return = mv_weights@temp_df.iloc[0].to_numpy().T
    mv_return = mv_weights@trade_day_y.to_numpy().T
    mv_daily_return = np.append(mv_daily_return,mv_return)

    # Adopt Better Prediction from Maximum Sharpe Ratio Portfolio or Minimum Volatility Portfolio
    if (ms_expected_return>mv_expected_return):
      daily_return = np.append(daily_return,ms_return)
    else:
      daily_return = np.append(daily_return,mv_return)

In [ ]:
# Ready for Result Exhibition
ms_total_return = np.prod(ms_daily_return+1)-1
mv_total_return = np.prod(mv_daily_return+1)-1
total_return = np.prod(daily_return+1)-1
SnP_array = np.array(SnP_Return["SnP Log Return"].loc[trade_days])
SnP_total_return = np.prod(SnP_array+1)-1

ms_cum_return = np.cumprod(ms_daily_return+1)
mv_cum_return = np.cumprod(mv_daily_return+1)
cum_return = np.cumprod(daily_return+1)
SnP_cum_return = np.cumprod(SnP_array+1)

print(ms_cum_return[-1], mv_cum_return[-1], cum_return[-1], SnP_cum_return[-1])

In [ ]:
# Calculate Information Ratio
avg_return_algorithm = np.array(daily_return).mean()
avg_return_SnP = SnP_array.mean()
tracking_error = np.std(np.array(daily_return)-SnP_array,ddof=1)
IR = (avg_return_algorithm-avg_return_SnP)/tracking_error
annual_IR = (avg_return_algorithm-avg_return_SnP)/tracking_error*252**(1/2)
print(IR,annual_IR)

# Result Exhibition

In [ ]:
plt.style.use('ggplot')
sns.set(rc={'figure.figsize':(11.7,8.27)})

plt.xlabel("Date")
plt.ylabel("Return")
plt.title("Portfolios performance")
plt.legend(loc="upper left")
plt.plot(trade_days,ms_cum_return,label="Maximised Sharpe Ratio")
plt.plot(trade_days,mv_cum_return,label="Minimised volatility")
plt.plot(trade_days,cum_return,label="Algorithm")
plt.plot(trade_days,SnP_cum_return,c="Red",label="S&P500 Index")
plt.legend(loc="upper left")

plt.show()

In [ ]:
result = pd.DataFrame({'ms_daily_return':ms_daily_return,'mv_daily_return':mv_daily_return,'algo_daily_return':daily_return,'SnP_daily_return':SnP_array})
result.index = trade_days

In [ ]:
from google.colab import  drive

drive.mount('/drive',force_remount=True)
result.to_csv('/drive/My Drive/result_20220510_2302.csv')